In [1]:
#change working directory
import os
print(os.getcwd())
os.chdir('/home/student/A2/drive-download-20240507T073745Z-001')

/home/student/A2


In [2]:
"""
Part b
Task 1. Processing Data Stream (45%)
Question a
"""
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = "172.16.33.120"

client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

2024-01-01 00:00:00


In [3]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json
import pandas as pd

hostip = ipadd # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer 
    
def publish_msg(producer, topic, data):
    try:
        #print(data)        #send the  value by dumps to JSON object
                            #then encode with utf-8 into bytes
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [4]:
"""
Event Producer 1
"""
climate_stream_csv = pd.read_csv("./climate_streaming.csv", sep=',')
climate_stream_csv.head()

,latitude,longitude,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
0,-37.623,149.323,19,56.8,7.9,11.1,0.00I,154
1,-38.038,142.986,15,50.7,9.2,13.0,0.02G,128
2,-37.950,142.366,16,53.6,8.1,15.0,0.00G,133
3,-38.231,147.172,24,61.6,7.7,14.0,0.00I,186
4,-37.903,145.250,24,62.3,7.0,13.0,0.00I,185


In [ ]:
"""
Main Function
"""

if __name__ == '__main__':
    topic = 'Producer1'
    print('Publishing records..')
    producer01 = connect_kafka_producer()


    # Assuming 'climate' is your DataFrame
    #print(len(climate.index))
    rand_row_indices = list(range(len(climate_stream_csv.index)))
    random.shuffle(rand_row_indices)  
    #print(len(indices))

    for index, climate_row in climate_stream_csv.iterrows():
        climate_row = climate_stream_csv.loc[rand_row_indices[index]]

        #inner-join
    #     if fires_list:
            #print(fires_list)

        #left-outer join
        #print(start_date.strftime('%Y-%m-%d'))
        climate_doc = {
                                        #string from time with format, 
                                        #since MongoDB can not store object
            #.strftime('%d/%m/%Y')
            #convert datetime object to string in format
            # laster dumps not work on datetime object but number of string
            'latitude': float(climate_row['latitude']),
            'longitude': float(climate_row['longitude']),
            'air_temperature_celcius': int(climate_row['air_temperature_celcius']),
            'relative_humidity': float(climate_row['relative_humidity']),
            'windspeed_knots': float(climate_row['windspeed_knots']),
            'max_wind_speed': float(climate_row['max_wind_speed']),
            #'precipitation ' blank space count as char in csv column
            # get to do try except, return None if column not exist
            #'precipitation ': str(climate_row.get(['precipitation '])),
            #'precipitation': str(climate_row.get(['precipitation'])),
            'precipitation': str(climate_row['precipitation']),
            'GHI_w/m2': int(climate_row['GHI_w/m2']),
            'producer': "Producer_1",
            'date': start_date.strftime('%Y-%m-%d'),
            'created_time': int(10)

        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        print(climate_doc)
        publish_msg(producer01, topic, climate_doc)
        start_date += timedelta(days=1)
        # change to 1 for debugging
        sleep(10)
        #sleep(10) # every 10 seconds send




Publishing records..
{'latitude': -37.591, 'longitude': 149.33, 'air_temperature_celcius': 16, 'relative_humidity': 46.7, 'windspeed_knots': 10.0, 'max_wind_speed': 16.9, 'precipitation': ' 0.00I', 'GHI_w/m2': 141, 'producer': 'Producer_1', 'date': '2024-01-01', 'created_time': 10}
{'latitude': -37.472, 'longitude': 148.119, 'air_temperature_celcius': 15, 'relative_humidity': 49.9, 'windspeed_knots': 7.9, 'max_wind_speed': 13.0, 'precipitation': ' 0.01G', 'GHI_w/m2': 129, 'producer': 'Producer_1', 'date': '2024-01-02', 'created_time': 10}
{'latitude': -36.293, 'longitude': 146.148, 'air_temperature_celcius': 16, 'relative_humidity': 50.8, 'windspeed_knots': 5.8, 'max_wind_speed': 12.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 136, 'producer': 'Producer_1', 'date': '2024-01-03', 'created_time': 10}
{'latitude': -37.602, 'longitude': 149.295, 'air_temperature_celcius': 17, 'relative_humidity': 50.8, 'windspeed_knots': 9.0, 'max_wind_speed': 15.9, 'precipitation': ' 0.00G', 'GHI_w/m2': 145,

{'latitude': -36.764, 'longitude': 144.165, 'air_temperature_celcius': 14, 'relative_humidity': 52.4, 'windspeed_knots': 6.7, 'max_wind_speed': 11.1, 'precipitation': ' 0.00I', 'GHI_w/m2': 118, 'producer': 'Producer_1', 'date': '2024-02-02', 'created_time': 10}
{'latitude': -38.0427, 'longitude': 141.0271, 'air_temperature_celcius': 19, 'relative_humidity': 52.9, 'windspeed_knots': 8.1, 'max_wind_speed': 15.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 159, 'producer': 'Producer_1', 'date': '2024-02-03', 'created_time': 10}
{'latitude': -37.692, 'longitude': 143.593, 'air_temperature_celcius': 14, 'relative_humidity': 45.8, 'windspeed_knots': 6.5, 'max_wind_speed': 9.9, 'precipitation': ' 0.00G', 'GHI_w/m2': 124, 'producer': 'Producer_1', 'date': '2024-02-04', 'created_time': 10}
{'latitude': -37.1875, 'longitude': 146.8024, 'air_temperature_celcius': 28, 'relative_humidity': 58.3, 'windspeed_knots': 9.3, 'max_wind_speed': 15.9, 'precipitation': ' 0.00I', 'GHI_w/m2': 223, 'producer': 'Prod

{'latitude': -37.469, 'longitude': 148.103, 'air_temperature_celcius': 10, 'relative_humidity': 47.1, 'windspeed_knots': 4.6, 'max_wind_speed': 8.9, 'precipitation': ' 0.00G', 'GHI_w/m2': 88, 'producer': 'Producer_1', 'date': '2024-03-05', 'created_time': 10}
{'latitude': -37.401, 'longitude': 148.067, 'air_temperature_celcius': 13, 'relative_humidity': 42.5, 'windspeed_knots': 14.0, 'max_wind_speed': 19.0, 'precipitation': ' 0.04G', 'GHI_w/m2': 118, 'producer': 'Producer_1', 'date': '2024-03-06', 'created_time': 10}
{'latitude': -37.69, 'longitude': 143.605, 'air_temperature_celcius': 14, 'relative_humidity': 45.7, 'windspeed_knots': 9.2, 'max_wind_speed': 15.9, 'precipitation': ' 0.00I', 'GHI_w/m2': 124, 'producer': 'Producer_1', 'date': '2024-03-07', 'created_time': 10}
{'latitude': -36.4274, 'longitude': 142.1944, 'air_temperature_celcius': 22, 'relative_humidity': 46.9, 'windspeed_knots': 12.9, 'max_wind_speed': 19.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 193, 'producer': 'Produc

{'latitude': -37.61, 'longitude': 149.296, 'air_temperature_celcius': 16, 'relative_humidity': 48.3, 'windspeed_knots': 14.4, 'max_wind_speed': 19.0, 'precipitation': ' 0.00G', 'GHI_w/m2': 139, 'producer': 'Producer_1', 'date': '2024-04-06', 'created_time': 10}
{'latitude': -37.862, 'longitude': 144.175, 'air_temperature_celcius': 20, 'relative_humidity': 52.9, 'windspeed_knots': 17.0, 'max_wind_speed': 21.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 167, 'producer': 'Producer_1', 'date': '2024-04-07', 'created_time': 10}
{'latitude': -37.618, 'longitude': 149.281, 'air_temperature_celcius': 19, 'relative_humidity': 53.8, 'windspeed_knots': 9.2, 'max_wind_speed': 15.0, 'precipitation': ' 0.02G', 'GHI_w/m2': 158, 'producer': 'Producer_1', 'date': '2024-04-08', 'created_time': 10}
{'latitude': -37.856, 'longitude': 143.416, 'air_temperature_celcius': 15, 'relative_humidity': 53.7, 'windspeed_knots': 6.8, 'max_wind_speed': 11.1, 'precipitation': ' 0.51G', 'GHI_w/m2': 125, 'producer': 'Produc

{'latitude': -37.459, 'longitude': 148.092, 'air_temperature_celcius': 13, 'relative_humidity': 48.5, 'windspeed_knots': 7.1, 'max_wind_speed': 15.9, 'precipitation': ' 0.00G', 'GHI_w/m2': 113, 'producer': 'Producer_1', 'date': '2024-05-08', 'created_time': 10}
{'latitude': -37.635, 'longitude': 149.303, 'air_temperature_celcius': 24, 'relative_humidity': 53.4, 'windspeed_knots': 9.9, 'max_wind_speed': 15.9, 'precipitation': ' 0.00I', 'GHI_w/m2': 200, 'producer': 'Producer_1', 'date': '2024-05-09', 'created_time': 10}
{'latitude': -37.013, 'longitude': 141.5355, 'air_temperature_celcius': 10, 'relative_humidity': 44.1, 'windspeed_knots': 7.3, 'max_wind_speed': 12.0, 'precipitation': ' 0.39G', 'GHI_w/m2': 90, 'producer': 'Producer_1', 'date': '2024-05-10', 'created_time': 10}
{'latitude': -36.3769, 'longitude': 144.7505, 'air_temperature_celcius': 14, 'relative_humidity': 50.9, 'windspeed_knots': 5.9, 'max_wind_speed': 13.0, 'precipitation': ' 0.00I', 'GHI_w/m2': 119, 'producer': 'Produ

In [ ]:
# """
# Main Function
# """
# if __name__ == '__main__':
#     topic = 'Scenaries01'
#     print('Publishing records..')
#     producer01 = connect_kafka_producer()


#     # Assuming 'climate' is your DataFrame
#     #print(len(climate.index))
#     rand_row_indices = list(range(len(climate.index)))
#     random.shuffle(rand_row_indices)  
#     #print(len(indices))

#     for index, climate_row in climate.iterrows():
#         climate_row = climate.loc[rand_row_indices[index]]

#         #inner-join
#     #     if fires_list:
#             #print(fires_list)

#         #left-outer join
#         #print(start_date.strftime('%Y-%m-%d'))
#         climate_doc = {
#             'stations': str(climate_row['station']),
#                                         #string from time with format, 
#                                         #since MongoDB can not store object
#             #.strftime('%d/%m/%Y')
#             #convert datetime object to string in format
#             # laster dumps not work on datetime object but number of string
#             'date': start_date.strftime('%Y-%m-%d'),
#             'air_temperature_celcius': int(climate_row['air_temperature_celcius']),
#             'relative_humidity': float(climate_row['relative_humidity']),
#             'windspeed_knots': float(climate_row['windspeed_knots']),
#             'max_wind_speed': float(climate_row['max_wind_speed']),
#             'precipitation': str(climate_row['precipitation']),
#             'GHI_w/m2': int(climate_row['GHI_w/m2']),
#             'producer': "Producer_1"

#         }
#         #update start date
#         #print(climate_doc['date'])
#         #producer publish msg in strictured streaming
#         # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
#         # msg = json.dumps(climate_doc)
#         print(climate_doc)
#         publish_msg(producer01, topic, climate_doc)
#         start_date += timedelta(days=1)
#         # change to 1 for debugging
#         sleep(1)
#         #sleep(10) # every 10 seconds send


